In [51]:
%%bash
pip install 'great_expectations'

In [52]:
import os
import pandas as pd
import great_expectations as gx
from great_expectations.data_context import FileDataContext

In [ ]:
%%bash
pip install streamlit

In [ ]:
import streamlit as st
import numpy as np
from io import StringIO

In [ ]:
st.title("Freddie Mac Dataset Quality Assessment App")

# File upload
uploaded_file = st.file_uploader("Upload a CSV or XLS file", type=["csv", "xls"])

# User selects data type
data_type = st.radio("Select Data Type:", ("Origination Data", "Monthly Performance Data"))

if uploaded_file is not None:
    # Load data into Pandas DataFrame
    if data_type == "Origination Data":
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_excel(uploaded_file)

In [53]:
path_to_repo_dir="/Users/anamikabharali/Documents/UNI/BD/streamlit" #local path to csv
path_to_data_dir=f"{path_to_repo_dir}/gx/data" 
expectation_suite_name="GE_Suite" 

In [54]:
context = FileDataContext.create(project_root_dir=path_to_repo_dir)

In [55]:
os.mkdir(path_to_data_dir)

df = pd.read_csv('/Users/anamikabharali/Documents/UNI/BD/streamlit/historical_data_2022Q1.txt', delimiter="|", header = None)



In [56]:
if data_type == "Origination Data":
    df.columns=['Credit_Score', 'First_Payment_Date', 'First_Time_Homebuyer_Flag','Maturity_Date','MSA', 'MI','Number_of_Units','Occupancy_Status','CLTV','DTI','UPB','LTV','Original_Interest_Rate','Channel','PPM','Amortization','Property_State','Property_Type','Postal_Code','Loan_Sequence_Number','Loan_Purpose','Original_Loan_Term','Number_of_Borrowers','Seller_Name','Servicer_Name','Super_Conforming_Flag','HARP','Program_Indicator','HARP_Indicator','Property_Valuation_Method','nterest_Only','Mortgage_Insurance_Cancellation_Indicator']
else:
    df.columns=['Loan_Sequence_Number','Monthly_Reporting_Period','Current_Actual_UPB','Current_Loan_Delinquency_Status','Loan_Age','Remaining_Months_to_Legal_Maturity','Defect_Settlement_Date','Modification_Flag','Zero_Balance_Code','Zero_Balance_Effective_Date','Current_Interest_Rate','Current_Deferred_UPB','Due_Date_of_Last_Paid_Installment_(DDLPI)','MI_Recoveries','Net_Sales_Proceeds','Non_MI_Recoveries','Expenses','Legal_Costs','Maintenance_and_Preservation_Costs','Taxes_and_Insurance','Miscellaneous_Expenses','Actual_Loss_Calculation','Modification_Cost','Step_Modification_Flag','Deferred_Payment_Plan','Estimated_Loan_to_Value_(ELTV)','Zero_Balance_Removal_UPB','Delinquent_Accrued_Interest','Delinquency_Due_to_Disaster','Borrower_Assistance_Status_Code','Current_Month_Modification_Cost','Interest_Bearing_UPB']

# Save the DataFrame as a CSV file
df.to_csv(f"{path_to_repo_dir}/gx/data/historical_data_2022Q1.csv", index=False)

In [57]:
df.head()

,Loan_Sequence_Number,Monthly_Reporting_Period,Current_Actual_UPB,Current_Loan_Delinquency_Status,Loan_Age,Remaining_Months_to_Legal_Maturity,Defect_Settlement_Date,Modification_Flag,Zero_Balance_Code,Zero_Balance_Effective_Date,...,Modification_Cost,Step_Modification_Flag,Deferred_Payment_Plan,Estimated_Loan_to_Value_(ELTV),Zero_Balance_Removal_UPB,Delinquent_Accrued_Interest,Delinquency_Due_to_Disaster,Borrower_Assistance_Status_Code,Current_Month_Modification_Cost,Interest_Bearing_UPB
0,763,202203,N,205202,47664.0,0,1,P,95,47,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
1,691,202203,N,203702,47664.0,0,1,P,67,25,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,678,202203,N,203702,NaN,0,2,P,28,44,...,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,7
3,802,202203,N,205202,33460.0,0,1,P,75,27,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,767,202203,N,205202,NaN,0,1,P,55,44,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [ ]:
#df = df.drop('ColumnName', axis=1)
df.head()

In [58]:
# Give your Datasource a name
datasource_name = "Local_FileSystem_Source"
datasource = context.sources.add_pandas_filesystem(name=datasource_name, base_directory=path_to_data_dir)

# Give your first Asset a name
asset_name = "Freddie_Mac"
batching_regex = r"historical_data_\d{4}Q\d.csv" #change this

asset = datasource.add_csv_asset(name=asset_name,batching_regex=batching_regex)

# Build batch request
batch_request = asset.build_batch_request()

Create expectations suite

In [59]:
data_asset = context.get_datasource(datasource_name).get_asset(asset_name)
batch_request = data_asset.build_batch_request()

In [60]:
context.list_expectation_suite_names()

[]

In [61]:
context. add_or_update_expectation_suite(expectation_suite_name)

{
  "expectation_suite_name": "GE_Suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.17.19"
  }
}

In [62]:
validator = context.get_validator(batch_request=batch_request, expectation_suite_name=expectation_suite_name)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Loan_Sequence_Number,Monthly_Reporting_Period,Current_Actual_UPB,Current_Loan_Delinquency_Status,Loan_Age,Remaining_Months_to_Legal_Maturity,Defect_Settlement_Date,Modification_Flag,Zero_Balance_Code,Zero_Balance_Effective_Date,...,Modification_Cost,Step_Modification_Flag,Deferred_Payment_Plan,Estimated_Loan_to_Value_(ELTV),Zero_Balance_Removal_UPB,Delinquent_Accrued_Interest,Delinquency_Due_to_Disaster,Borrower_Assistance_Status_Code,Current_Month_Modification_Cost,Interest_Bearing_UPB
0,763,202203,N,205202,47664.0,0,1,P,95,47,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
1,691,202203,N,203702,47664.0,0,1,P,67,25,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,678,202203,N,203702,NaN,0,2,P,28,44,...,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,7
3,802,202203,N,205202,33460.0,0,1,P,75,27,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,767,202203,N,205202,NaN,0,1,P,55,44,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


Validators for schema

In [ ]:
def validate_ge_origination():

    #schema check

    validator.expect_column_values_to_be_of_type("First_Payment_Date", "datetime")
    validator.expect_column_values_to_be_of_type("First_Time_Homebuyer_Flag", "str")
    validator.expect_column_values_to_be_of_type("Maturity_Date", "datetime")
    validator.expect_column_values_to_be_of_type("MSA", "int")
    validator.expect_column_values_to_be_of_type("MI", "int")
    validator.expect_column_values_to_be_of_type("Number_of_Units", "int")
    validator.expect_column_values_to_be_of_type("Occupancy_Status", "str")
    validator.expect_column_values_to_be_of_type("CLTV", "float")
    validator.expect_column_values_to_be_of_type("DTI", "float")
    validator.expect_column_values_to_be_of_type("UPB", "int")
    validator.expect_column_values_to_be_of_type("LTV", "float")
    validator.expect_column_values_to_be_of_type("Original_Interest_Rate", "float")

    validator.expect_column_values_to_be_of_type("Channel", "str")

    validator.expect_column_values_to_be_of_type("PPM", "str")

    validator.expect_column_values_to_be_of_type("Amortization", "str")

    validator.expect_column_values_to_be_of_type("Property_State", "str")

    validator.expect_column_values_to_be_of_type("Property_Type", "str")

    validator.expect_column_values_to_be_of_type("Postal_Code", "int")

    validator.expect_column_values_to_be_of_type("Loan_Sequence_Number", "str")

    validator.expect_column_values_to_be_of_type("Loan_Purpose", "str")

    validator.expect_column_values_to_be_of_type("Original_Loan_Term", "int")

    validator.expect_column_values_to_be_of_type("Number_of_Borrowers", "int")

    validator.expect_column_values_to_be_of_type("Seller_Name", "str")

    validator.expect_column_values_to_be_of_type("Servicer_Name", "str")

    validator.expect_column_values_to_be_of_type("Super_Conforming_Flag", "str")

    validator.expect_column_values_to_be_of_type("HARP", "str")

    validator.expect_column_values_to_be_of_type("Program_Indicator", "str")

    validator.expect_column_values_to_be_of_type("HARP_Indicator", "str")

    validator.expect_column_values_to_be_of_type("Property_Valuation_Method", "int")

    validator.expect_column_values_to_be_of_type("nterest_Only", "str")

    validator.expect_column_values_to_be_of_type("Mortgage_Insurance_Cancellation_Indicator", "str")

 

 

    #Validation

    validator.expect_column_values_to_be_in_set ("First_Time_Homebuyer_Flag", ["Y","N"])

    validator.expect_column_values_to_be_in_set("Occupancy_Status", ["P", "I", "S"])

    validator.expect_column_values_to_be_in_set("Channel", ["B", "R", "C"])

    validator.expect_column_values_to_match_regex("Loan_Sequence_Number", regex=r'^P\d{2}Qn[a-zA-Z0-9]{7}$')

    validator.expect_column_values_to_match_regex("HARP_Indicator", regex=r'^P\d{2}Qn[a-zA-Z0-9]{7}$')

    validator.expect_column_value_lengths_to_be_between("Credit_Score", min_value=1, max_value=4)

    validator.expect_column_value_lengths_to_be_between("UPB", min_value=1, max_value=12)

    validator.expect_column_values_to_be_in_set("Amortization", ["FRM", "ARM"])

    validator.expect_column_values_to_match_regex("Postal_Code", r"^\d{5}$")

    validator.expect_column_values_to_match_regex("Original_Interest_Rate", regex=r'^\d{3}\.\d{3}$')

    expected_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

    validator.expect_column_values_to_be_in_set("Property_State", value_set=expected_states)

    

    #Missing data

    for column in df.columns:

        validator.expect_column_values_to_not_be_null(column)

 

 

    validator.save_expectation_suite()

 



In [63]:
def validate_ge_monthly():
    #Schema Check
    validator.expect_column_values_to_be_of_type("Loan_Sequence_Number", "str")
    validator.expect_column_values_to_be_of_type("Monthly_Reporting_Period", "datetime")
    validator.expect_column_values_to_be_of_type("Current_Actual_UPB", "float")
    validator.expect_column_values_to_be_of_type("Current_Loan_Delinquency_Status", "str")
    validator.expect_column_values_to_be_of_type("Loan_Age", "int")
    validator.expect_column_values_to_be_of_type("Remaining_Months_to_Legal_Maturity", "int")
    validator.expect_column_values_to_be_of_type("Defect_Settlement_Date", "datetime")
    validator.expect_column_values_to_be_of_type("Modification_Flag", "str")
    validator.expect_column_values_to_be_of_type("Zero_Balance_Code", "int")
    validator.expect_column_values_to_be_of_type("Zero_Balance_Effective_Date", "datetime")
    validator.expect_column_values_to_be_of_type("Current_Interest_Rate", "float")
    validator.expect_column_values_to_be_of_type("Current_Deferred_UPB", "int")
    validator.expect_column_values_to_be_of_type("Due_Date_of_Last_Paid_Installment_(DDLPI)", "datetime")
    validator.expect_column_values_to_be_of_type("MI_Recoveries", "float")
    validator.expect_column_values_to_be_of_type("Net_Sales_Proceeds", "str")
    validator.expect_column_values_to_be_of_type("Non_MI_Recoveries", "float")
    validator.expect_column_values_to_be_of_type("Expenses", "float")
    validator.expect_column_values_to_be_of_type("Legal_Costs", "float")
    validator.expect_column_values_to_be_of_type("Maintenance_and_Preservation_Costs", "float")
    validator.expect_column_values_to_be_of_type("Taxes_and_Insurance", "float")
    validator.expect_column_values_to_be_of_type("Miscellaneous_Expenses", "float")
    validator.expect_column_values_to_be_of_type("Actual_Loss_Calculation", "float")
    validator.expect_column_values_to_be_of_type("Modification_Cost", "float")
    validator.expect_column_values_to_be_of_type("Step_Modification_Flag", "str")
    validator.expect_column_values_to_be_of_type("Deferred_Payment_Plan", "str")
    validator.expect_column_values_to_be_of_type("Estimated_Loan_to_Value_(ELTV)", "int")
    validator.expect_column_values_to_be_of_type("Zero_Balance_Removal_UPB", "float")
    validator.expect_column_values_to_be_of_type("Delinquent_Accrued_Interest", "float")
    validator.expect_column_values_to_be_of_type("Delinquency_Due_to_Disaster", "str")
    validator.expect_column_values_to_be_of_type("Borrower_Assistance_Status_Code", "str")
    validator.expect_column_values_to_be_of_type("Current_Month_Modification_Cost", "float")
    validator.expect_column_values_to_be_of_type("Interest_Bearing_UPB", "float")

    #other validations
    validator.expect_column_values_to_be_unique("Loan_Sequence_Number")  #Loan seq number will be unique
    validator.expect_column_values_to_match_regex("Loan_Sequence_Number", regex=r'^P\d{2}Qn[a-zA-Z0-9]{7}$')
    validator.expect_column_values_to_match_regex("Current_Actual_UPB", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Current_Interest_Rate", regex=r'^\d{5}\.\d{3}$')
    validator.expect_column_values_to_match_regex("MI_Recoveries", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Non_MI_Recoveries", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Expenses", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Legal_Costs", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Maintenance_and_Preservation_Costs", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Miscellaneous_Expenses", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Taxes_and_Insurance", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Actual_Loss_Calculation", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Modification_Cost", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Zero_Balance_Removal_UPB", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Delinquent_Accrued_Interest", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Current_Month_Modification_Cost", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_values_to_match_regex("Interest_Bearing_UPB", regex=r'^\d{10}\.\d{2}$')
    validator.expect_column_value_lengths_to_be_between("Zero_Balance_Removal_UPB", min_value=1, max_value=12)
    validator.expect_column_value_lengths_to_be_between("Current_Deferred_UPB", min_value=1, max_value=12)
    validator.expect_column_values_to_be_in_set("MI_Recoveries", ["B", "R", "C"])
    validator.expect_column_values_to_be_in_set("Modification_Flag", ["I", "P", "S"])
    

    #For null value check
    for column in df.columns:
        validator.expect_column_values_to_not_be_null(column)

    validator.save_expectation_suite()

In [ ]:
if data_type == "Origination Data":
    validate_ge_origination()
else:
    validate_ge_monthly()

In [64]:
validate_ge_monthly()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Create Checkpoint

In [65]:
checkpoint = context.add_or_update_checkpoint(
    name="Freddie_Mac_Monthly_v1",
    validator=validator
)

In [66]:
checkpoint_result = checkpoint.run(run_name="Manual_run")

Calculating Metrics:   0%|          | 0/192 [00:00<?, ?it/s]

In [67]:
context. build_data_docs()

{'local_site': 'file:///Users/anamikabharali/Documents/UNI/BD/streamlit/gx/uncommitted/data_docs/local_site/index.html'}